# Trajectory Prediction with LSTM
### CS230 - Deep Learning -  Final Submission. 
#### Mitchell Dawson, Benjamin Goeing, Tyler Hughes.  

## Project Introduction
 In this project, we train a number of sequential models to predict the trajectory of pedestrians as they move through a scene and interact with one another. Our model could be used to predict movements of crowds of people and vehicles given an overhead image of a scene. This may have potential applications in helping to make public spaces less susceptible to crowding or accidents, improving control of autonomous vehicles, and video surveillance.  We provide several sources of additional input to the sequential models to improve its performance.  For example, information related to the tracked subject's location and trajectory, the scene layout, and the presence of other people.


## Dataset

We are leveraging a part of the Stanford Drone dataset, which contains a large number of overhead images of crowded spaces on Stanford campus. (http://cvgl.stanford.edu/projects/uav_data/). Here are some examples of drone footage from this dataset:

<img src="images/bookstore.jpg?raw=true" width="250"/>  <img src="images/deathCircle.jpg?raw=true" width="250"/>   

Each scene in the dataset also contains csv files with rows in the form of (f,o,x,y).  Here 'f' is the frame number, 'o' is the unique object identifier (which person is being tracked) and x,y are the x and y coordinates (not normalized) of the object.

The part of this dataset we use was recently released as part of a benchmark competition (http://trajnet.stanford.edu). The data made available for the competition is a cleaned up version of the full Stanford Drone dataset. It consists of 30 scenes from 6 different locations around Stanford's campus. The data for each scene consists of one image of representative frame of the video recorded for that scene and a csv file like that described above. The scenes contain 8494 pedestrian trajectories in total.

We divided the dataset into train, development, and test sets in a roughly 70/15/15 split, with the train set containing 22 scenes, the development set containing 4 scenes, and the test set containing 4 scenese.

For our first models, the data required minimal preprocessing. We normalized the x,y coordinates of all pedestrians so that they were in the range [-1, 1]. The original x,y coordinates were contained in different ranges in different scenes, so this was necessary to make different scenes comparable.

In later models we tried, we wanted to incorporate additional information about the scene into our inputs. In particular, extracted the locations of all other pedestrians (i.e. not the target pedestrian) in every frame and constructed occupancy grids. We also segmented the image for each scene and constructed similar grids representing the terrain in each scene. This is explained in more detail below. All preprocessing code is contained the included in the files file processing.py and image_label_script.m.

We also augmented our dataset by flipping and rotating the scenes, to create 7 new scenes  for each scene. Having 8 times more trajectories in our training set dramatically improved the performance of our models.

## The Prediction Task

Our objective is predict the last 10 locations of a pedestrian's trajectory given the first 10 locations. A stylized example of this is depicted below.

<img src="images/test_predict.png?raw=true" width="150">   

Our loss function is the total Euclidean distance between the predicted locations and the ground truth points:

$$\mathcal{L} = \sum_{j=1}^N \sqrt{\big(x_j - \bar{x}_j\big)^2 + \big(y_j - \bar{y}_j\big)^2}  $$

Here, $j$ is the frame number, $x$ and $y$ are the ground truth coordinates, and $\bar{x}$ and $\bar{y}$ are the predicted coordinates.

## Module Imports and Data Loading

Our models are implemented in pytorch. We import that here, along with some other dependencies, the code for our models, and some helper functions. We also load a small subset of our test set that we'll use to demonstrate our models.

In [4]:
# Module import (add here as necessary)
import numpy as np
import torch
import torch.utils.data

from gpu_lstm import TrajectoryPredictor
from gpu_lstm_vel import TrajectoryPredictorVel
from gpu_linear_lstm_vel import TrajectoryPredictorLinearVel

from plot_trajectories import plot_trajectories
from linear_error import compute_linear_error

test_trajectories = np.load('gates_1.npy')
data = test_trajectories[:, :10, :]
target = test_trajectories[:, 10:, :]
data_tensor = torch.from_numpy(data).float().cuda()
target_tensor = torch.from_numpy(target).float().cuda()
dataset = torch.utils.data.TensorDataset(data_tensor, target_tensor)

## Our First Model: A Vanilla LSTM

We first implemented a plain LSTM model with the x,y coordinates in each frame as the input, and the predicted x,y coordinates as the output.

<img src="images/LSTM_xy.png?raw=true" width="500">   

During training, the model (and all subsequent models) take the ground truth for the first 10 frames as its first 10 inputs. The output produced on the 10th input is then treated as the prediction for the 11th frame. The ground truth for the second 10 frames is given as the next 10 inputs, with each output considered to be the prediction of the next frame (the last output is ignored). The loss is computed on the predicted locations. During testing, the first 10 inputs are the same, but the predicted locations are given as input starting with the 11th input. The last output is ignored and the loss is computed on the 10 predicted locations.

We trained the model with the Adam optimizer and found best performance with a learning rate of 0.002, Adam's default beta parameters, and no weight decay. We used batch sizes of 200 and trained for 120 epochs. Our model achieved average losses (average losses over all trajectories) of 0.4284, 0.3622, and 0.3852 on the train, dev, and test sets, respectively.

We load a pretrained version of the model and plot a few predicted trajectories.

In [1]:
# p = TrajectoryPredictor(2, 2, batch_size)
# p.train(train_loader, num_epochs)

# LOAD ONLY XY MODEL HERE!!!
# p_xy = ...

In [7]:
p_xy.test(dev_loader)

mean loss: 0.534271395503


We compare with the predictions of a linear model on the same trajectory, where the (x,y) coordinates of a future time frame are estimated by extrapolating from the person's velocity at the final observation time frame.

In [4]:
compute_linear_error(dev_trajectories, Nf)

mean loss: 0.128018440883


Clearly we have to include more information in order to improve this model.

## Velocity

We noticed that the trajectories predicted by the first model were generally linear when the ground truth trajectory was linear, the "heading" of the predicted trajectory was consistently off. This caused error to accumulate, as small differences in the direction of early predictions led to large error at later predictions. In an attempt to bias our model against this, we decided to add as an input the subject's estimated velocity at the 10th frame (i.e. the frame immediately before prediction). We also experimented with updating the estimating the velocity at every time step during prediction, but we found this caused errors to compound even more quickly.

We estimate the velocity with a simple finite difference formula, where $N$ is the final training frame and $\Delta t$ is the frame rate

$$v_x = \frac{x_N - x_{N-1}}{\Delta t}$$


$$v_y = \frac{y_N - y_{N-1}}{\Delta t}$$

We implemented a new model that incorporated this velocity calculation and used it as another input at each timestep. We also trained this model with the Adam optimizer, and found that similar hyperparameters worked well: a learning rate of 0.002, Adam's default beta parameters, and no weight decay. Again, we used batch sizes of 200 and trained for 120 epochs. Our model achieved average losses (average losses over all trajectories) of 0.3707, 0.3361, and 0.3519 on the train, dev, and test sets, respectively. This was a moderate improvement over our first model.

In [ ]:
# LOAD VELOCITY DATA IN:
# p_xyv = load_v
# PREDICT ON DEV SET

While this model performed better than with just $x$ and $y$, we still have some ways to go

## Dense Layer

Even with velocity as an additional input, the performance of our model was lacking, with even the train error relatively high. We figured that increasing the complexity of the model might lead to better performance. In order to add dimensionality to our LSTM and also allow for the model to learn an embedding for the input vectors, we added a dense layer between our input data and our LSTM input:

<img src="images/LSTM_xyv_dense.png?raw=true" width="500">   

This dense layer takes the 4 dimensional input and transforms it to an $LSTM_{size}$ dimensional input.  This intermediate dimension was a new tunable parameter, and found that performance was generally best with $LSTM_{size} = 20$.

We also trained this model with the Adam optimizer and found best performance with a learning rate of 0.001, Adam's default beta parameters, and no weight decay. We used batch sizes of 200 and trained for 140 epochs. Our model achieved average losses (average losses over all trajectories) of 0.3621, 0.3215, and 0.3496 on the train, dev, and test sets, respectively. This was a small and relatively insignificant improvement.

In [ ]:
# LOAD VELOCITY+DENSE DATA IN:
# p_xyv_dense = load_v
# PREDICT ON DEV SET

## Image Segmentation and Scene Information

We hoped to improve performance by incorporating additional information about the scene into our inputs. Specifically, we wanted to included information regarding where other pedestrians were in the scene and information regarding the types of terrain in the scene. The thinking behind this was that our model could perhaps learn, for example, that pedestrians tend to stay a certain distance away from each other and avoid colliding; or that pedestrains tend to walk straight on sidewalks and avoid the middle of the road.

To incorporate the information about the locations of other people in the frame, we constructed occupancy grids for each frame. Each occupancy grid was an $N_o \times N_o$ array whose entries are 0 or 1 depending on if there is pedestrian at the corresponding locations.

We incorporated terrain information by segmented the images that accompanied each scene in the dataset. We used MIT's LabelMe (http://labelme.csail.mit.edu/Release3.0/) tool to manually segment each image and classify each pixel by class:
- a) road
- b) sidewalk
- c) grass
- d) inaccessible (describing objects such as building walls, trees etc.) 

We wrote a MATLAB script to then clean up the labels output by LabelMe by ensuring that there were no overlaps or unlabelled pixels. With the resulting per-pixel labels, we created arrays similar to the occupancy grids that contained the terrain information at a lower resolution. Specifically, the arrays were $N_t \time N_t$ arrays where each entry was the plurality class label (1, 2, 3, or 4) at the pixels corresponding to that entry.

Here is an example of the scene labeling and segmentation process:

<img src="images/imageSeg.png?raw=true" width="250">

## Convolutional Neural Network Approach

We tried feeding the occupancy grids and terrain grids directly into our models, but we ran into memory constraints. Grids of any significant resolution simply took up to much space to feed in with every trajectory.

Therefore, we decided that the best approach would be to feed the information into the LSTM after being processed through a CNN.  

We used the middle layers of a pretrained network, AlexNet https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf, as our CNN, as it has already learned to extract useful features from image data.

In addition to the segmented background image and the occupancy grid, we also fed the CNN in an array representing where the target pedestrain (whose location we're predicting) is in the scene as our third channel.  This was to ensure that the spatial relationship between the tracked person and this other information was encoded in the same way by the CNN.

<img src="images/LSTM_full.png?raw=true" width="500">

This model ended up not being able to learn (as in, during training the loss never decreased), and we're not entirely sure why. Our hypothesis is that the grids we fed into the CNN were still too coarse-grained to contain any useful information.

## Discussion
 
#### Challenges

One of the major challenges we faced was our inability to get the model with CNN-encoded inputs to learn. We examined gradients, tried using different numbers of pretrained middle layers, and tried feeding higher resolution occupancy and terrain grids into the CNN, but nothing worked. This was disappointing given our high hopes for the model and the time it took to construct and encode the grids.


#### Evaluation of Results and Error Analysis

The Stanford Computational Vision and Geometry Lab, who created the dataset we used, reports a mean average displacement loss of about 30 pixels. As each scene has a different resolution, it is somewhat difficult to put this in an absolute comparison. However, based on a discussion we had with them, we can assume an average number of 1200 pixel per axis, which translates into a loss of 0.25. Comparing their loss of 0.25 to our loss of 0.04 shows that our model is performing fairly well, but could be further improved.

The primary source of error in our model remains the difficulty it has in handling straight trajectories, as those make up the bulk of the dataset. It is unclear to us what deep learning techniques we could use to alleviate this source of error. Perhaps rather than having only a deep learning model predicting trajectories, an ensemble of models (including very simple models that just extrapolate linearly from a pedestrian's trajectory) could all predict at each time step and give confidences in their prediction. The final prediction would then be chosen from among these in some fashion. In this case, the deep learning models might be called upon to only predict the most complex trajectories, something such models are more suited for.

#### Future Works

We think it might be useful for future work to explore different CNN architectures for encoding the scene and occupancy data.  We used a pre-trained model because our previous sections were already very computationally intensive, but perhaps training the CNN in conjunction with the LSTM would give better results.  

Also, since our segmented image array elements are just indices corresponding to the scene 'class' of each pixel, there could be several approaches to better use this data.  For example, we could construct separate binary arrays for each of the classes (for example, a separate 'inaccessable' array to feed to the CNN).

## Conclusion

In conclusion, we have shown that an RNN model can perform reasonably well on trajectory prediction tasks.  We have learned a great deal about how important it is to choose wisely when designing your inputs for the LSTM task.  If the input is too simple then the model will not have enough information to predict trajectories well.  On the other hand, throwing too much data at your LSTM will dramatically increase training time and computational cost, and is not even guarenteed to improve results.  We spent a good amount of time on this project trying to find the middle ground between these two extremes and had some limited success.  Although we did not end up beating the state of the art on these tasks, it was a fruitful learning experience.